In [9]:
import pandas as pd
import numpy as np

In [1]:
from os import walk
import os

path = '/'.join(os.path.abspath(os.getcwd()).split('/')[:-1]) + '/data/partitions/'

f = []

for (dirpath, dirnames, filenames) in walk(path):
    f.append(dirnames)



In [5]:
files = f[0]

In [6]:
def entropy(s):
  return np.sum(np.log(s**2))


def get_features(df:any, second:int) -> list:
    k = (df[df['second']==second]
    .loc[:,['x','y']]
    .kurt(axis=0)
    .to_dict())

    v = (df[df['second']==second]
    .loc[:,['x','y']]
    .kurt(axis=0)
    .to_dict())

    s = (df[df['second']==second]
    .loc[:,['x','y']]
    .skew(axis=0)
    .to_dict())

    e = (df[df['second']==second]
    .loc[:,['x','y']]
    .apply(lambda x: entropy(x),axis=0)
    .to_dict())

    results = [
        v['x'],v['y'],
        s['x'],s['y'],
        k['x'],k['y'],
        e['x'],e['y'],
        v['x'] + e['x'], v['y'] + e['y'],
        k['x'] + e['x'], k['y'] + e['y'],
        s['x'] + e['x'], s['y'] + e['y'],
        v['x'] + s['x'], v['y'] + s['y'],
        v['x'] + k['x'], v['y'] + k['y'],
        k['x'] + s['x'], k['y'] + s['y']
    ]

    return results

In [7]:
def pipeline(file:str) -> list:
    cols = ['var_x','var_y',
    'skew_x','skew_y',
    'kur_x','kur_y',
    'entropy_x','entropy_y',
    'var_x+entropy_x','var_y+entropy_y',
    'kur_x+entropy_x','kur_y+entropy_y',
    'skew_x+entropy_x','skew_y+entropy_y',
    'var_x+skew_x', 'var_y+skew_y',
    'var_x+skew_x', 'var_y+skew_y',
    'kur_x+skew_x', 'kur_y+skew_y']
    
    label = file.split('_')[1]

    df = pd.read_parquet(f'../data/partitions/{file}/')

    r = []
    for second in df['second'].unique():
        r.append(get_features(df, second))

    d = pd.DataFrame(data=r,columns=cols).mean().to_dict()
    d['label'] = label

    

    return d

In [10]:
from joblib import Parallel, delayed


results = Parallel(n_jobs=-1)(delayed(pipeline)(file) for file in files)

/home/davi/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
df = pd.DataFrame(results)

In [12]:
df.to_parquet('dataset.parquet')